In [9]:
import requests
import json
class ProdSysApi:
    
    def __init__(self, url_list, url_single, user, password):
        self._auth = requests.auth.HTTPBasicAuth(user, password)
        self._urls = [url_single, url_list]
        
    def api_url(self, id=None):
        if id is None:
            return self._urls[1]
        else:
            return self._urls[0].format(id)
    
    def list(self):
        response = requests.get(self.api_url(), auth=self._auth)
        print(len(response.content))
        data = response.json()
        return data
        
    def get(self, id):
        response = requests.get(self.api_url(id), auth=self._auth)
        data = response.json()
        return data
    
    def new(self, data):
            return requests.post(
                self.api_url(), 
                auth = self._auth,
                json = data,
            )
        
    def patch(self, id, data):
        if 'api/indesign/' in self._urls[0]:
            return requests.post(
                self.api_url(id), 
                auth = self._auth,
                data = {'json': json.dumps(data)},
            )
        else:
            return requests.patch(
                self.api_url(id), 
                auth = self._auth,
                json = data,
            )
            

        
old_api = ProdSysApi(
    user='api',
    password='Avier1gi',
    url_list = 'http://universitas.no/admin/api/indesign/json/',
    url_single = 'http://universitas.no/admin/api/indesign/{}/json/',
)
new_api = ProdSysApi(
    user='api',
    password='Avier1gi',
    url_list = 'http://universitas.no/api/legacy/?limit=500',
    url_single = 'http://universitas.no/api/legacy/{}/',
)
local_api = ProdSysApi(
    user='api',
    password='api',
    url_list = 'http://web:8000/api/legacy/?limit=10',
    url_single = 'http://web:8000/api/legacy/{}/',
)



def get_story_type(prodsys_mappe):
    """ Find correct story type for the story based on legacy label. """
    try:
        story_type = StoryType.objects.filter(prodsys_mappe=prodsys_mappe)[0]
    except IndexError:
        generic_section, exists = Section.objects.get_or_create(
            title='New Section')
        story_type = StoryType.objects.create(
            prodsys_mappe=prodsys_mappe,
            name="New Story Type ({label})".format(label=prodsys_mappe),
            section=generic_section,
        )
    # finally:
    return story_type


items = new_api.list()['results']
print([item['prodsak_id'] for item in items])

401683
[60641, 60677, 60706, 60710, 60693, 60740, 60736, 60734, 60733, 60722, 60999, 61075, 61080, 61163, 61316, 61630, 61636, 61640, 61645, 61675, 61731, 61732, 61770, 61828, 61839, 62009, 62012, 62011, 62056, 62057, 62072, 62073, 62099, 62119, 62175, 62196, 62200, 62224, 59771, 62299, 62312, 62316, 62321, 62322, 62323, 62326, 62313, 62334, 62397, 62398, 62407, 62468, 62472, 62487, 62491, 62505, 61935, 60731, 61089, 61078, 61934, 61628, 61077, 61632, 61796, 61936, 60848, 62173, 61845, 61234, 62168, 62172, 60671, 61464, 61150, 62452, 62436, 62453, 60956, 61949, 61789, 62166, 61631, 62041, 61878, 60577, 60966, 60989, 61011, 61018, 61032, 61031, 61071, 61072, 61142, 61033, 61611, 61651, 61652, 61653, 61691, 61704, 61705, 61720, 61733, 61735, 61738, 61739, 61737, 61769, 61771, 61787, 61843, 61914, 61937, 61942, 61941, 61969, 62033, 62165, 62179, 62220, 62251, 62288, 62292, 62373]


In [ ]:
for prodsak in old_api.list():
    print('{prodsak_id} {mappe} {arbeidstittel}'.format(**prodsak))
    get_story_type(prodsak['mappe'])
    
#def foo():
    response = local_api.new(prodsak)
    status = response.status_code
    print(status, response.content)
    assert status in [201, 400]

In [2]:
flags = {
    '1': Story.STATUS_JOURNALIST,
    '2': Story.STATUS_SUBEDITOR,
    '4': Story.STATUS_EDITOR,
}
from pprint import pprint
from datetime import datetime
import pytz
from functools import partial
def parser(value):
    dt=datetime.strptime(value, '%Y-%m-%d %H:%M:%S')
    return pytz.timezone('Europe/Oslo').localize(dt)


In [4]:
def pre_save(model_instance, add):
    """ Returns field's value just before saving. """
    return getattr(model_instance, 'modified')

Story._meta.get_field('modified').pre_save = pre_save

for story in Story.objects.filter(publication_status__lt=Story.STATUS_PUBLISHED):
    data = old_api.get(story.pk)
    created = parser(data.get('dato'))
    modified = parser(data.get('version_date'))
    story.created = created
    story.modified = modified
    story.publication_status = flags[data.get('flagg')]
    story.save(update_fields=['created', 'modified'] )
    print(story)
    print(created, modified, story.modified, sep='\n')

(10 drikkestopp på 10 trikkestopp - velkomstbilag)
2017-08-07 17:54:21+02:00
2017-08-07 17:54:21+02:00
2017-08-07 17:54:21+02:00
(NETT Anm Louien)
2017-08-14 16:17:18+02:00
2017-08-14 16:17:18+02:00
2017-08-14 16:17:18+02:00
(Kulturnotiser)
2017-08-14 17:31:03+02:00
2017-08-15 21:49:36+02:00
2017-08-15 21:49:36+02:00
(16 Leder)
2017-08-15 21:47:30+02:00
2017-08-15 21:47:30+02:00
2017-08-15 21:47:30+02:00
(NETT Anm Nils Bech)
2017-08-14 16:28:04+02:00
2017-08-15 22:53:18+02:00
2017-08-15 22:53:18+02:00
(16 Tomme SiO-hybler)
2017-08-15 18:09:17+02:00
2017-08-15 23:29:11+02:00
2017-08-15 23:29:11+02:00
(Åpningsseremonier)
2017-08-15 20:11:25+02:00
2017-08-15 20:11:25+02:00
2017-08-15 20:11:25+02:00
(Ad notam 16)
2017-08-15 19:08:01+02:00
2017-08-15 19:08:01+02:00
2017-08-15 19:08:01+02:00
(16 nyh Rettssak)
2017-08-15 20:14:49+02:00
2017-08-16 00:09:15+02:00
2017-08-16 00:09:15+02:00
(Tips til ferske studenter)
2017-08-15 23:01:52+02:00
2017-08-15 23:38:11+02:00
2017-08-15 23:38:11+02:00
(

In [ ]:
import pytz
pytz.country_timezones('no')

In [ ]:
testsak =saksliste2[1]
testsak.pop('prodsak_id', '')
testsak['mappe'] = 'kultur'
# pprint(testsak)
response = local_api.new(testsak)
pprint(response.json())

In [ ]:
Story.objects.filter(publication_status__lte=1).delete()

In [ ]:
response = api.post(id=23215, data={'arbeidstittel': 'awesome', 'produsert': 0})
print(response.request.headers, response.request.body, sep='\n')
print(response.url, response.status_code, response.text)
testsak = api.get(23215)
print(testsak['arbeidstittel'], testsak['version_no'], testsak['produsert'])

In [ ]:
from django.contrib.auth.models import Permission
pp = Permission.objects.filter(codename__contains='_story')
for p in pp: print(p.codename)jjj

In [ ]:
from django.contrib.auth import authenticate
create_user = get_user_model().objects.create
get_user_model().objects.filter(username='api').delete()
api_user = get_user_model().objects.create_user(
    username='api',
    password='api',
    email='api@api.com',
)
print(api_user)
login = authenticate(username='api', password='api')
print(login)

In [ ]:
from django.contrib.auth import get_user_model

def api_user():
    User = get_user_model()
    try:
        user = User.objects.get(username='api')
    except User.DoesNotExist:
        user = User.objects.create_user(
            username='api',
            password='api',
            email='api@example.com',
        )
    permissions = list(Permission.objects.filter(codename__contains='_story'))
    user.user_permissions.add(*permissions)
    for p in permissions:
        print(p.codename)
    return user

api_user()

In [ ]:
saker = api.list()
til_web = list(filter(lambda sak: sak['produsert'] == '6', saker))
pprint(til_web[7])


In [ ]:
pprint(api.get(23194))

In [ ]:
def deserialize_story(story):
    output = [] 
    head_tags = [('tit', 'title'),('ing', 'lede'),('tema', 'theme_word'),('stikktit', 'kicker')]
    for tag, attr in head_tags:
        text = getattr(story, attr)
        if text:
            output.append(f'@{tag}:{text}')
    for bl in story.byline_set.select_related('contributor'):
        output.append(str(bl))
    output.append(story.bodytext_markup) 
    for aside in story.asides():
        output.append(aside.child.bodytext_markup)
    for pullquote in story.pullquotes():
        output.append(pullquote.child.bodytext_markup)
    return '\n'.join(output)
    
story = Story.objects.order_by('?').last()
txt1 = deserialize_story(story)
prodstory = api.get(story.prodsak_id)
txt2 = prodstory.get('tekst')
print(story.prodsak_id)
print(txt1)
print(txt2)

In [ ]:
story = Story.objects.get(id=61090)
story.images()

In [ ]:
imgstories = Story.objects.filter(pk__in=StoryImage.objects.all().values_list('parent_story', flat=True).distinct())
imgstories[0].pk
